### Jacopo Sala 
### Download time series of HYCOM forcing at the location of the selected events

In [2]:
import xarray as xr
import csv
import pandas as pd
import pickle
import numpy as np
import os
from datetime import date, timedelta, datetime
# from datetime import datetime as dtime
import sys
import matplotlib.pyplot as plt
import netCDF4 as nc
import glob
import os.path
import webbrowser
import time
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import matplotlib as mpl
from FUNCTIONS_HYCOM import set_regions, set_regions_tags, find_timedelta64_index
import pickle as pkl
import subprocess

In [3]:
# define start and end years
# start_year = 2011
# end_year = 2015
start_date = '2011-01-01'
end_date = '2015-12-31'

#start_date = '2010-12-25'
#end_date = '2011-01-01'

# start_date = '2015-09-02'
# end_date = '2015-11-30'

# for each element in list, we have FILENAME, VARS and TYPE
url_info = [['precip', 'var=precip', 'sea'],\
            ['TaqaQrQp', 'var=airtmp&var=vapmix&var=radflx&var=shwflx', 'sea'],\
            ['surtmp', 'var=surtmp', 'sea'],\
            ['uv-10m', 'var=wndnwd&var=wndewd', 'sec2'],\
            ['wndspd', 'var=wndspd', 'sec2'],\
            ['sfcprs', 'var=airprs', 'sec'],\
            ['dswsfc', 'var=dswflx', 'sea'],\
            ['dlwsfc', 'var=dlwflx', 'sea']]

# url_info = [['sfcprs', 'var=airprs', 'sec'],\
#             ['dswsfc', 'var=dswflx', 'sea'],\
#             ['dlwsfc', 'var=dlwflx', 'sea']]
# url_info = [['precip', 'var=precip', 'sea'],\
#             ['TaqaQrQp', 'var=airtmp&var=vapmix&var=radflx&var=shwflx', 'sea'],\
#             ['surtmp', 'var=surtmp', 'sea'],\
#             ['uv-10m', 'var=wndnwd&var=wndewd', 'sec2'],\
#             ['wndspd', 'var=wndspd', 'sec2']]

# HYCOM_forcing_save_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/HYCOM_FORCING_for_each_event/'
HYCOM_forcing_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/HYCOM_FORCING/REGIONS/'
# download_dir = '/Users/jacoposala/Downloads/'

url_prefix = 'https://ncss.hycom.org/thredds/ncss/datasets/force/ncep_cfsv2/netcdf/'
file_prefix = 'cfsv2-TYPE_YEAR_01hr_FILENAME.nc?VARS&north=NORTH&west=WEST&east=EAST&south=SOUTH&disableProjSubset=on&horizStride=1&time_start=TIMESTART&time_end=TIMEEND&timeStride=1'
file_suffix = '&addLatLon=true&accept=netcdf4'

file_extension = '.nc4'


In [4]:
# # Define function to download forcing data

# def download_forcing_data(date, north, south, east, west, url_prefix, file_prefix, file_suffix, f_dir, file_extension):
#     date_to_download = pd.to_datetime(date)
#     year = str(date_to_download)[0:4]
#     month = str(date_to_download)[5:7]
#     day = str(date_to_download)[8:10]
#     hour = str(date_to_download)[11:13]
#     date = datetime(int(year), int(month), int(day), int(hour))
#     year = f"{int(date_to_download.year):04d}"
#     month = f"{int(date_to_download.month):02d}"
#     day = f"{int(date_to_download.day):02d}"
    
#     # The year at the beginning of the URL is not always consistent with the actual year of the data being downloaded:
#     # Each year "folder" includes files starts at 12 UTC on Jan 1, and goes to 9 UTC of Jan 1 of the following year
#     if (date_to_download.month == 1) & (date_to_download.day == 1) & (date_to_download.hour < 12):
#         year_prefix = f"{int(date_to_download.year - 1):04d}"
#     else:
#         year_prefix = year

#     # Fill out the URL now
#     file_suffix_hh = file_suffix.replace('hh', hour, 1)
#     file_prefix_coord = file_prefix.replace('NORTH', north, 1)
#     file_prefix_coord = file_prefix_coord.replace('SOUTH', south, 1)
#     file_prefix_coord = file_prefix_coord.replace('EAST', east, 1)
#     file_prefix_coord = file_prefix_coord.replace('WEST', west, 1)
    
#     # Assemble the URL
#     url = '"' + url_prefix + file_prefix_coord + year + '-' + month + '-' + day + file_suffix_hh + '"'
    
#     name_download = file_prefix_coord + year + '-' + month + '-' + day + file_suffix_hh
#     name_download = name_download.replace('%3A', ':', 2)
#     print(url)
#     print(name_download)
#     # Download file
#     if not os.path.exists(HYCOM_focring_save_dir + file_prefix_coord + year + month + day + file_suffix_hh + file_extension):
#         # Download file
#         !wget {url}
#         # Wait until the file is downloaded
#         while not os.path.exists(name_download):
#             time.sleep(1)
#         print('Download done!')
#         # ...then rename and move file
#         if os.path.isfile(name_download):
#             os.rename(name_download, HYCOM_focring_save_dir + file_prefix_coord + year + month + day + file_suffix_hh + file_extension)
#         else:
#             print('Missing ' + name_download)
#         print('Renaming or warning done!')
#     # Open the file
#     dset_day = xr.open_dataset(f'{f_dir}{file_prefix_coord}{year}{month}{day}{file_suffix_hh}{file_extension}')
#     # Remove the file
#     #if os.path.isfile(f'{f_dir}{file_prefix}{year}{month}{day}{file_suffix_hh}{file_extension}'):
#     #    os.remove(f'{f_dir}{file_prefix}{year}{month}{day}{file_suffix_hh}{file_extension}')
#     print('Dataset open - ready to return')
#     return(dset_day)


In [5]:
# cd /Users/jacoposala/Downloads/

In [6]:
# Generate a list of dates
date_list_start = pd.date_range(start=start_date, end=end_date)

# Print the list of dates
# print(date_list_start)

In [7]:
# Generate a list of dates
date_list_end = pd.date_range(start=start_date + ' 23:00:00', end=end_date + ' 23:00:00')

# Print the list of dates
# print(date_list_end)

In [8]:
# List all the regions

# lat_max, lat_min, lon_max, lon_min for every region
regions = set_regions()
regions_tags = set_regions_tags(regions)
print(regions_tags)

['WPacific_2of3']


In [9]:
for r,r_tag in zip(regions, regions_tags):
    lat_min = r[1]
    lat_max = r[0]
    lon_min = r[3]
    lon_max = r[2]
#     time.sleep(30)
    print(r_tag)
    for itime_start, itime_end in zip(date_list_start, date_list_end):
#         time.sleep(1)
        for iinfo in url_info:
            url_mid = file_prefix.replace('TYPE', str(iinfo[2]))\
            .replace('YEAR', str(itime_start.year))\
            .replace('FILENAME', str(iinfo[0]))\
            .replace('VARS', str(iinfo[1]))\
            .replace('NORTH', str(lat_max))\
            .replace('WEST', str(lon_min))\
            .replace('EAST', str(lon_max))\
            .replace('SOUTH', str(lat_min))\
            .replace('TIMESTART', itime_start.strftime('%Y-%m-%d %H:%M:%S').replace(' ','T').replace(':','%3A') + 'Z')\
            .replace('TIMEEND', itime_end.strftime('%Y-%m-%d %H:%M:%S').replace(' ','T').replace(':','%3A') + 'Z')
            
            url = url_prefix + url_mid + file_suffix
            
            subprocess.run(['wget', '-O' + HYCOM_forcing_dir + r_tag + '/' + r_tag + '_' + iinfo[0] + '_' + itime_start.strftime('%Y%m%d') + file_extension, url])
        if itime_start.day == 1 and itime_start.month == 1:
            time.sleep(3)
            print('Starting' + itime_start.strftime('%Y'))
            


WPacific_2of3


In [ ]:
ciao

In [ ]:
# LOAD high-speed event information (lat, lon, time)

# List all files across basins
# folder_tags = ['WPacific_3of3', 'Indian']
folder_tags = ['NorthAtlantic']#, 'EPacific', 'WPacific_1of3', 'WPacific_2of3', 'WPacific_3of3', 'SWPacific_1of2', 'SWPacific_2of2', 'Indian']
HYCOM_dir = '/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/DATA/6_HYCOM_near_TC_53x_pkl_NEW_BOX_postproc_/'
# timesteps_per_day = 8
number_of_days_before_event = 6
number_of_days_after_event = 24

files = []

for i in np.arange(len(folder_tags)):

    HYCOM_dir_current = HYCOM_dir.replace('postproc_', 'postproc_' + str(folder_tags[i])) 
    files_in_dir = [j for j in sorted(glob.glob(HYCOM_dir_current + '*MLD.pkl'))]
    files = files + files_in_dir
    
# Load first to initialize variables that will store them all
new_dict = pkl.load(open(files[1], "rb"))
# Add info from the rest
datasets = []
index = [] 
N = 0
for ifile in np.arange(len(files)):
    df = pkl.load(open(files[ifile], "rb"))
    # 
    _, index = np.unique(df['time'], return_index=True)
    df_unique_time = df.isel(time=index)
    date_of_event = pd.to_datetime(df.sel_date.values, format='%Y-%m-%d %H:%M:%S').to_datetime64()
    start_date_event = date_of_event - np.timedelta64(number_of_days_before_event, 'D')
    end_date_event = date_of_event + np.timedelta64(number_of_days_after_event, 'D')

    # Create new time axis (hourly)
    time_new = pd.date_range(start_date_event, end_date_event, freq='1h')

#         time_new = pd.date_range(df.time.values[0], df.time.values[0] + np.timedelta64(int(len(df.time.values)/timesteps_per_day), 'D'), freq='1h')
    # Interpolation using new time axis just created
    df_unique_time_interp = df_unique_time.interp(time=time_new)
#         if len(df_unique_time_interp.time.values):
#             print('Check what is wrong')
#             ciao1
#         print(df.time.values)
#         df['time'] = time_new

        
    df_unique_time_interp["index_tag"] = files[ifile].split("/")[-1].split("event")[0] + 'event_' + \
    files[ifile].split("/")[-1].split("event")[-1].split("_")[1]
    
    df_unique_time_interp['time_event_original'] = df_unique_time_interp.time
    # save date_of_event
    df_unique_time_interp['date_of_event'] = date_of_event

    df_unique_time_interp = df_unique_time_interp.assign_coords(time=df_unique_time_interp.time.values - date_of_event)
    df_unique_time_interp = df_unique_time_interp.assign_coords(index=N)
    df_unique_time_interp = df_unique_time_interp.expand_dims('index')
    N = N+1
    
    df_unique_time_interp = df_unique_time_interp.rename_dims({'time': 'delta_time'})
    df_unique_time_interp = df_unique_time_interp.rename({'time': 'delta_time'})
    
    
#     df_unique_time_interp = df_unique_time_interp.drop_dims('time')
    datasets.append(df_unique_time_interp)
    
print('before combined') 

combined = xr.concat(datasets, dim='index')




In [ ]:
# Check: if the minimum it is not True: PROBLEM 
min(np.abs(combined.time_event_original.isel(delta_time=6*24).values - combined.date_of_event.values)) == 0

In [ ]:
combined.to_netcdf('combined.nc4')

sst = 
http://ncss.hycom.org/thredds/ncss/datasets/force/ncep_cfsv2/netcdf/cfsv2-sea_2011_01hr_surtmp.nc?var=surtmp&north=NORTH&west=WEST&east=EAST&south=SOUTH&disableProjSubset=on&horizStride=1&time_start=2011-01-01T00%3A00%3A00Z&time_end=2011-01-01T23%3A00%3A00Z&timeStride=1&addLatLon=true&accept=netcdf4

precip = 
http://ncss.hycom.org/thredds/ncss/datasets/force/ncep_cfsv2/netcdf/cfsv2-sea_2011_01hr_precip.nc?var=precip&north=NORTH&west=WEST&east=EAST&south=SOUTH&disableProjSubset=on&horizStride=1&time_start=2011-01-01T00%3A00%3A00Z&time_end=2011-01-01T23%3A00%3A00Z&timeStride=1&addLatLon=true&accept=netcdf4

surface downwelling shortwave flux = 
http://ncss.hycom.org/thredds/ncss/datasets/force/ncep_cfsv2/netcdf/cfsv2-sea_2011_01hr_dswsfc.nc?var=dswflx&north=NORTH&west=WEST&east=EAST&south=SOUTH&disableProjSubset=on&horizStride=1&time_start=2011-01-01T00%3A00%3A00Z&time_end=2011-01-01T23%3A00%3A00Z&timeStride=1&addLatLon=true&accept=netcdf4

surface downwelling longwave flux = 
http://ncss.hycom.org/thredds/ncss/datasets/force/ncep_cfsv2/netcdf/cfsv2-sea_2011_01hr_dlwsfc.nc?var=dlwflx&north=NORTH&west=WEST&east=EAST&south=SOUTH&disableProjSubset=on&horizStride=1&time_start=2011-01-01T00%3A00%3A00Z&time_end=2011-01-01T23%3A00%3A00Z&timeStride=1&addLatLon=true&accept=netcdf4

air temperature &specific humidity & surface net downward radiative flux & surface net downward shortwave flux= http://ncss.hycom.org/thredds/ncss/datasets/force/ncep_cfsv2/netcdf/cfsv2-sea_2011_01hr_TaqaQrQp.nc?var=airtmp&var=radflx&var=shwflx&var=vapmix&north=NORTH&west=WEST&east=EAST&south=SOUTH&disableProjSubset=on&horizStride=1&time_start=2011-01-01T00%3A00%3A00Z&time_end=2011-01-01T23%3A00%3A00Z&timeStride=1&addLatLon=true&accept=netcdf4

eastward wind & northward wind = 
http://ncss.hycom.org/thredds/ncss/datasets/force/ncep_cfsv2/netcdf/cfsv2-sec2_2011_01hr_uv-10m.nc?var=wndewd&var=wndnwd&north=NORTH&west=WEST&east=EAST&south=SOUTH&disableProjSubset=on&horizStride=1&time_start=2011-01-01T00%3A00%3A00Z&time_end=2011-01-01T23%3A00%3A00Z&timeStride=1&addLatLon=true&accept=netcdf4

wind speed = 
http://ncss.hycom.org/thredds/ncss/datasets/force/ncep_cfsv2/netcdf/cfsv2-sec2_2011_01hr_wndspd.nc?var=wndspd&north=NORTH&west=WEST&east=EAST&south=SOUTH&disableProjSubset=on&horizStride=1&time_start=2011-01-01T00%3A00%3A00Z&time_end=2011-01-01T23%3A00%3A00Z&timeStride=1&addLatLon=true&accept=netcdf4

    
    
    
    